In [ ]:
# import packages
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
# activate the driver
driver = webdriver.Chrome()

In [ ]:
# url I want to scrape
url = 'https://www.basketball-reference.com/boxscores/197606040BOS.html'
# open the url
driver.get(url)

In [ ]:
# save the webpage source code
src = driver.page_source

In [ ]:
driver.quit()

In [ ]:
# create Beautiful Soup object
parser = BeautifulSoup(src, 'lxml')

In [ ]:
# pull all boxscores on the webpage
stat_tables = parser.findAll('table', attrs = {'class': 'sortable stats_table now_sortable'})
len(stat_tables)

In [ ]:
# skip first row (overheader)
rows = stat_tables[0].findAll('tr')[1:]

In [ ]:
# create a list with the column headers
headers = rows[0].findAll('th')
headerlist = [h.text.strip() for h in headers]

In [ ]:
headerlist

In [ ]:
# skip the header row
data = rows[1:]
# get names column
player_names = [row.find('th').text.strip() for row in rows]
# get player stats
player_stats = [[stat.text.strip() for stat in row.findAll('td')] for row in data]

In [ ]:
for i in range(len(player_stats)):
    # ignore header with i+1, insert player names into the beginning of each data row
    player_stats[i].insert(0, player_names[i+1])

# create player stats dataframe
player_box_df = pd.DataFrame(player_stats, columns=headerlist)
# drop 'Reserves' row
player_box_df.drop(player_box_df[player_box_df['Starters'] == 'Reserves'].index, inplace=True)
player_box_df.rename(columns={'Starters':'Players'}, inplace=True)

In [ ]:
player_box_df